<h1><span class="burk">Things that remind to use this tool</h1></span>

<h4> Change the name of (input) file</h4>
<h4> Change the name of PD's team</h4>
<h4> <span class="burk">Do not change</span> the 'PDS info' file columns</h4>
<h4> In the input file, <span class="burk">the name of columns must be match</span> with this below of the code</h4>

# Import & Load files 

In [20]:
# =========================================================== IMPORT LI ===============================================
!pip install xlrd==1.2.0
historical_s = '/user/hk-pdops-svc/Tools/LUNA/pdf_allocation_info.xlsx'
historical_f = '/user/hk-pdops-svc/Tools/LUNA/thai_test.xlsx'
historical_l = '/user/hk-pdops-svc/Tools/LUNA/lang_dict.xlsx'
historical_result='/user/hk-pdops-svc/Tools/LUNA/'
# =========================================================== HDFS READER ===============================================
from zapp.hadoop import impala_client
query_client = impala_client.ImpalaClient()
from zapp.hadoop.webhdfsext import HdfsClient
hdfs_client = HdfsClient(user_name='hk-pdops-svc')

Looking in indexes: https://repo-hkg.agodadev.io/api/pypi/agoda-pypi-virtual/simple
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


# Reading files

In [27]:
# Reading both pds's info and user's input files
hdfs_client.download_file(historical_s, './pdf_allocation_info.xlsx')
hdfs_client.download_file(historical_f, './thai_test.xlsx')
hdfs_client.download_file(historical_l, './lang_dict.xlsx')
# Import Library
import sys
import openpyxl
from openpyxl import load_workbook
import statistics
pds_info = openpyxl.load_workbook('pdf_allocation_info.xlsx')
input_file = openpyxl.load_workbook('thai_test.xlsx')
lang_file = openpyxl.load_workbook('lang_dict.xlsx')

# Get sheets
sheet_input = input_file.active
sheet_info = pds_info.active
sheet_lang = lang_file.active
# Get max rows from each of files
maxRow_input = sheet_input.max_row
maxRow_info = sheet_info.max_row
maxRow_lang = sheet_lang.max_row
# Get language dictionary file
lang_dict = {}
for i in range(2, maxRow_lang+1):
    lang_dict[sheet_lang.cell(i,1).value] = sheet_lang.cell(i,2).value

start download from /user/hk-pdops-svc/Tools/LUNA/pdf_allocation_info.xlsx to ./pdf_allocation_info.xlsx
downloaded ./pdf_allocation_info.xlsx
start download from /user/hk-pdops-svc/Tools/LUNA/thai_test.xlsx to ./thai_test.xlsx
downloaded ./thai_test.xlsx
start download from /user/hk-pdops-svc/Tools/LUNA/lang_dict.xlsx to ./lang_dict.xlsx
downloaded ./lang_dict.xlsx


# Select a team

In [28]:
input_team='Thailand'

# Reading PDS info file

In [29]:
emps=[]
emps_cnt=[]
region_base=[]
country_base=[]
lang1=[]
lang2=[]
lang3=[]
lang4=[]
lang5=[]
timezones_min=[]
timezones_max=[]
region_base_cnt={}
country_base_cnt={}
timezones_cnt_min={}
timezones_cnt_max={}
lang1Cnt={}
lang2Cnt={}
lang3Cnt={}
lang4Cnt={}
lang5Cnt={}
emps_num=0

# Get PDS info
for i in range(2, maxRow_info+1):
    if sheet_info.cell(i,2).value == input_team:
        # PD's Email
        emps.append(sheet_info.cell(i,4).value)
        emps_cnt.append(0)
        
        # PD's region base
        tmpRegion=sheet_info.cell(i,3).value
        region_base.append(tmpRegion)
        tmpRegion_count=region_base_cnt.get(tmpRegion)
        if tmpRegion_count is None:
            region_base_cnt[tmpRegion]=1
        else:
            region_base_cnt[tmpRegion]=tmpRegion_count+1
            
        # PD's Country base
        tmpCountry=sheet_info.cell(i,6).value
        country_base.append(tmpCountry)
        tmpCountry_count=country_base_cnt.get(tmpCountry)
        if tmpCountry_count is None:
            country_base_cnt[tmpCountry]=1
        else:
            country_base_cnt[tmpCountry]=tmpCountry_count+1
            
        # Native language
        tmpLang1=sheet_info.cell(i,10).value
        lang1.append(tmpLang1)
        tmpLang1Cnt=lang1Cnt.get(tmpLang1)
        if tmpLang1Cnt is None:
            lang1Cnt[tmpLang1]=1
        else:
            lang1Cnt[tmpLang1]=tmpLang1Cnt+1
            
        # Second Language
        tmpLang2=sheet_info.cell(i,11).value
        lang2.append(tmpLang2)
        tmpLang2Cnt=lang2Cnt.get(tmpLang2)
        if tmpLang2Cnt is None:
            lang2Cnt[tmpLang2]=1
        else:
            lang2Cnt[tmpLang2]=tmpLang2Cnt+1
            
        # Third Language
        tmpLang3=sheet_info.cell(i,12).value
        lang3.append(tmpLang2)
        tmpLang3Cnt=lang3Cnt.get(tmpLang3)
        if tmpLang3Cnt is None:
            lang3Cnt[tmpLang3]=1
        else:
            lang3Cnt[tmpLang3]=tmpLang3Cnt+1
        
        # Fourth Language
        tmpLang4=sheet_info.cell(i,13).value
        lang4.append(tmpLang4)
        tmpLang4Cnt=lang4Cnt.get(tmpLang4)
        if tmpLang4Cnt is None:
            lang4Cnt[tmpLang4]=1
        else:
            lang4Cnt[tmpLang4]=tmpLang4Cnt+1
        
         # Fifth Language
        tmpLang5=sheet_info.cell(i,14).value
        lang5.append(tmpLang5)
        tmpLang5Cnt=lang5Cnt.get(tmpLang5)
        if tmpLang5Cnt is None:
            lang5Cnt[tmpLang5]=1
        else:
            lang5Cnt[tmpLang5]=tmpLang5Cnt+1
            
        # Longitude Min
        tmpTime_min=sheet_info.cell(i,17).value
        timezones_min.append(tmpTime_min)
        tmpTimeCnt_min=timezones_cnt_min.get(tmpTime_min)
        if tmpTimeCnt_min is None:
            timezones_cnt_min[tmpTime_min]=1
        else:
            timezones_cnt_min[tmpTime_min]=tmpTimeCnt_min+1
            
        # Longitude Max
        tmpTime_max=sheet_info.cell(i,18).value
        timezones_max.append(tmpTime_max)
        tmpTimeCnt_max=timezones_cnt_max.get(tmpTime_max)
        if tmpTimeCnt_max is None:
            timezones_cnt_max[tmpTime_max]=1
        else:
            timezones_cnt_max[tmpTime_max]=tmpTimeCnt_max+1
            
print("============PDS LISTS===============")
print(emps)
print(emps_cnt)
print("============Region & Country Base LISTS===============")
print(region_base)
print(country_base)
print(region_base_cnt)
print(country_base_cnt)
print("============LANGUAGE LISTS===============")
print(lang1)
print(lang2)
print(lang3)
print(lang4)
print(lang5)
print("============THE NUMBER OF LANGUAGE DICTIONARY===============")
print(lang1Cnt)
print(lang2Cnt)
print(lang3Cnt)
print(lang4Cnt)
print(lang5Cnt)
print("============TIMEZONES===============")
print(timezones_min)
print(timezones_max)

============PDS LISTS===============
['kwintra.burapaka@agoda.com', 'Malisa.Phoongoen@agoda.com', 'Chitchonnanee.Potipat@agoda.com', 'Kingrak.Nirachmongkol@agoda.com', 'Patinya.Makumleg@agoda.com', 'Patsachon.Niyomthai@agoda.com', 'Paweena.Somton@agoda.com', 'Pornpatra.Panyanonwat@agoda.com', 'Prangsai.Chaiatchariyakul@agoda.com', 'Suneeta.Ayae@agoda.com', 'Tappawat.Aumumphai@agoda.com', 'kultida.nantavisuth@agoda.com', 'natchaya.tanpattanatanakorn@agoda.com', 'nichakorn.cholsaranont@agoda.com', 'asri.manuthasna@agoda.com', 'pitchanee.mohpichai@agoda.com', 'jutatip.supathum@agoda.com', 'Tanthawan.Malone@agoda.com', 'Nata.Tanpanich@agoda.com']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
============Region & Country Base LISTS===============
['Kwintra Burapaka', 'Malisa Phoongoen', 'Chitchonnanee Potipat', 'Kingrak Nirachmongkol', 'Patinya Makumleg', 'Patsachon Niyomthai', 'Paweena Somton', 'Pornpatra Panyanonwat', 'Prangsai Chaiatchariyakul', 'Suneeta Ayae', 'Tappawat Aumu

# Reading user's input file

In [30]:
# Get column names INPUT
emps_num=len(emps)          
loop_cnt=0
country_index=-1
longitude_index=-1
pds_index=-1
mp_index=-1
num_of_pds = len(emps_cnt)
allocation = maxRow_input / num_of_pds
allocation = round(allocation)
while True:
    loop_cnt+=1
    if sheet_input.cell(1,loop_cnt).value=='country_name':
        country_index=loop_cnt
    if sheet_input.cell(1,loop_cnt).value=='longitude':
        longitude_index=loop_cnt  
    if sheet_input.cell(1,loop_cnt).value=='PDS':
        pds_index=loop_cnt  
    if sheet_input.cell(1,loop_cnt).value=='MP':
        mp_index=loop_cnt 
    if country_index > -1 and longitude_index > -1 and pds_index > -1 and mp_index > -1:
        break

# Sorting the Margin Point

In [31]:
mp_list = []
for i in range(2, maxRow_input+1):
    mp=sheet_input.cell(i,mp_index).value
    mp_list.append(sheet_input.cell(i,mp_index).value)
mp_list.sort(reverse = True)
print(mp_list)

[16154, 15361, 12890, 12674, 12239, 11256, 10930, 10450, 10187, 10142, 9341, 9221, 9198, 9177, 9137, 8970, 8894, 8850, 8767, 8728, 8696, 8634, 8529, 8445, 8422, 8386, 8316, 8125, 8118, 8044, 7969, 7903, 7839, 7811, 7809, 7807, 7790, 7776, 7717, 7705, 7698, 7572, 7463, 7443, 7441, 7419, 7363, 7288, 7279, 7236, 7209, 7187, 7173, 7015, 7008, 7007, 6986, 6980, 6951, 6938, 6926, 6856, 6850, 6796, 6787, 6728, 6718, 6679, 6667, 6578, 6575, 6555, 6539, 6480, 6476, 6475, 6463, 6442, 6440, 6432, 6415, 6408, 6391, 6359, 6333, 6326, 6308, 6229, 6218, 6218, 6214, 6198, 6189, 6155, 6142, 6124, 6123, 6120, 6087, 6082, 6030, 6021, 6020, 6015, 6007, 5993, 5976, 5969, 5942, 5933, 5930, 5909, 5855, 5832, 5808, 5799, 5797, 5792, 5780, 5768, 5763, 5714, 5713, 5677, 5661, 5646, 5641, 5634, 5624, 5608, 5572, 5570, 5554, 5553, 5504, 5483, 5482, 5479, 5476, 5475, 5449, 5448, 5448, 5445, 5439, 5420, 5399, 5394, 5375, 5369, 5364, 5347, 5346, 5316, 5300, 5293, 5286, 5271, 5253, 5234, 5231, 5221, 5194, 5188, 5175,

# The first loop for filtering based on 'Native Language'

In [32]:
# The first loop
for i in range(2, maxRow_input+1):
    country_name = sheet_input.cell(i,country_index).value
    lang_name = lang_dict[country_name]
    now_lang_cnt = lang1Cnt.get(lang_name)
    if now_lang_cnt is None:
        continue
    if now_lang_cnt==1:
        for j in range(0, emps_num):
            if lang1[j]==lang_name:
                sheet_input.cell(i,pds_index).value=emps[j]
                emps_cnt[j]+=1  
                        
    elif now_lang_cnt > 1:
        current_index = -1
        current_cnt = -1
        for j in range(0, emps_num):
            if (lang1[j]==lang_name and emps_cnt[j]<current_cnt) or (lang1[j]==lang_name and current_cnt<0):
                current_cnt = emps_cnt[j]
                current_index = j
        if current_index > -1:
            sheet_input.cell(i,pds_index).value=emps[current_index]
            emps_cnt[current_index]+=1

print(emps_cnt)

#input_file.save("myfile.xlsx")
# upload("myfile.xlsx", "hdfs://path")

[250, 250, 250, 250, 250, 250, 250, 250, 249, 249, 249, 249, 249, 249, 249, 249, 249, 249, 249]


# The second loop for filtering based on 'Second Language'

In [33]:
for i in range(2, maxRow_input+1):
    country_name = sheet_input.cell(i,country_index).value
    lang_name = lang_dict[country_name]
    now_lang_cnt2 = lang2Cnt.get(lang_name)
    if now_lang_cnt2 is None:
        continue
        
    if now_lang_cnt2==1:
        for j in range(0, emps_num):
            if lang2[j]==lang_name and emps_cnt[j] < allocation:
                sheet_input.cell(i,pds_index).value=emps[j]
                emps_cnt[j]+=1  
                
    if now_lang_cnt2 > 1:
        current_index = -1
        current_cnt = -1
        longitude = sheet_input.cell(i,longitude_index).value
        for j in range(0, emps_num):
            if (lang2[j]==lang_name) and (timezones_min[j] < longitude < timezones_max[j]) and (emps_cnt[j] < allocation): 
                if (lang2[j]==lang_name and emps_cnt[j]<current_cnt) or (lang2[j]==lang_name and current_cnt<0):
                    current_cnt = emps_cnt[j]
                    current_index = j
        if current_index > -1:
            sheet_input.cell(i,pds_index).value=emps[current_index]
            emps_cnt[current_index]+=1

print(emps_cnt)
print(allocation)
#input_file.save("myfile.xlsx")

[260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260]
260


# Third loop for filtering based on 'Third Language'

In [34]:
for i in range(2, maxRow_input+1):
    country_name = sheet_input.cell(i,country_index).value
    lang_name = lang_dict[country_name]
    now_lang_cnt3 = lang2Cnt.get(lang_name)
    if now_lang_cnt3 is None:
        continue
        
    if now_lang_cnt3==1:
        for j in range(0, emps_num):
            if lang3[j]==lang_name and emps_cnt[j] < allocation:
                mp_list.sort(reverse = True)
                sheet_input.cell(i,pds_index).value=emps[j]
                emps_cnt[j]+=1  
                
    elif now_lang_cnt3 > 1:
        current_index = -1
        current_cnt = -1
        longitude = sheet_input.cell(i,longitude_index).value
        for j in range(0, emps_num):
            if (lang3[j]==lang_name and emps_cnt[j]<current_cnt) or (lang3[j]==lang_name and current_cnt<0):
                if timezones_min[j] < longitude < timezones_max[j] :
                    mp_list.sort(reverse = True)
                    current_cnt = emps_cnt[j]
                    current_index = j
        if current_index > -1:
            mp_list.sort(reverse = True)
            sheet_input.cell(i,pds_index).value=emps[current_index]
            emps_cnt[current_index]+=1
            
print(emps_cnt)
print(allocation)

[271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 270, 270, 270, 270, 270]
260


# Fourth loop for filtering based on 'Fourth Language'

In [35]:
for i in range(2, maxRow_input+1):
    country_name = sheet_input.cell(i,country_index).value
    lang_name = lang_dict[country_name]
    now_lang_cnt4 = lang2Cnt.get(lang_name)
    if now_lang_cnt4 is None:
        continue
        
    if now_lang_cnt4==1:
        for j in range(0, emps_num):
            if lang4[j]==lang_name and emps_cnt[j] < allocation:
                mp_list.sort(reverse = True)
                sheet_input.cell(i,pds_index).value=emps[j]
                emps_cnt[j]+=1  
                
    elif now_lang_cnt4 > 1:
        current_index = -1
        current_cnt = -1
        longitude = sheet_input.cell(i,longitude_index).value
        for j in range(0, emps_num):
            if (lang4[j]==lang_name and emps_cnt[j]<current_cnt) or (lang4[j]==lang_name and current_cnt<0):
                if timezones_min[j] < longitude < timezones_max[j] :
                    mp_list.sort(reverse = True)
                    current_cnt = emps_cnt[j]
                    current_index = j
        if current_index > -1:
            mp_list.sort(reverse = True)
            sheet_input.cell(i,pds_index).value=emps[current_index]
            emps_cnt[current_index]+=1
            
print(emps_cnt)
print(allocation)

[271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 270, 270, 270, 270, 270]
260


# Fifth loop for filtering based on 'Fifth Language'

In [36]:
for i in range(2, maxRow_input+1):
    country_name = sheet_input.cell(i,country_index).value
    lang_name = lang_dict[country_name]
    now_lang_cnt5 = lang2Cnt.get(lang_name)
    if now_lang_cnt5 is None:
        continue
        
    if now_lang_cnt5==1:
        for j in range(0, emps_num):
            if lang4[j]==lang_name and emps_cnt[j] < allocation:
                mp_list.sort(reverse = True)
                sheet_input.cell(i,pds_index).value=emps[j]
                emps_cnt[j]+=1  
                
    elif now_lang_cnt5 > 1:
        current_index = -1
        current_cnt = -1
        longitude = sheet_input.cell(i,longitude_index).value
        for j in range(0, emps_num):
            if (lang5[j]==lang_name and emps_cnt[j]<current_cnt) or (lang5[j]==lang_name and current_cnt<0):
                if timezones_min[j] < longitude < timezones_max[j] :
                    mp_list.sort(reverse = True)
                    current_cnt = emps_cnt[j]
                    current_index = j
        if current_index > -1:
            mp_list.sort(reverse = True)
            sheet_input.cell(i,pds_index).value=emps[current_index]
            emps_cnt[current_index]+=1
            
print(emps_cnt)
print(allocation)

[271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 270, 270, 270, 270, 270]
260


In [37]:
%%bash

pwd

ls

/ssd/yarn/nm/usercache/AGODA+tphuong/appcache/application_1613465132458_376925/container_e159_1613465132458_376925_01_000002
container_tokens
lang_dict.xlsx
launch_container.sh
myfile.xlsx
pdf_allocation_info.xlsx
thai_test.xlsx
tmp


In [38]:
??hdfs_client

In [39]:
query_client._hdfs

Better HDFS (connected to it-research@namenode-1.hk.hadoop.agoda.local:14000)

# Saving the output file

In [17]:
hdfs_client.upload_file("myfile.xlsx", "/user/hk-pdops-svc/Tools/LUNA/myfile.xlsx", overwrite=True)

start upload to /user/hk-pdops-svc/Tools/LUNA/myfile.xlsx
uploaded /user/hk-pdops-svc/Tools/LUNA/myfile.xlsx


True

# Sending an email to Users

In [19]:
hdfs_client.download_file("/user/hk-pdops-svc/Tools/LUNA/myfile.xlsx", "./myfile.xlsx")
from zapp.mail import mail
mail.send_mail(
        send_from='nat.tran@agoda.com',
        send_to=['nat.tran@agoda.com','bruce.marks@agoda.com','dongkyu.lee@agoda.com'],
        subject='you received this because LUNA now can send attachment to users successfully',
        text="YAY",
        files=['myfile.xlsx'])

# import smtplib
# from os.path import basename
# from email.mime.application import MIMEApplication
# from email.mime.multipart import MIMEMultipart
# from email.mime.image import MIMEImage
# from email.mime.text import MIMEText
# from email.utils import COMMASPACE, formatdate

# mail_content = '''Hey,
# I can send email with attachment now. YAY!
# But why the file cannot be openned?
# :cry :cry :cry
# '''

# sender_address = 'dongkyu.lee@agoda.com'
# receiver_address = 'Chata.Salwala@agoda.com'

# message = MIMEMultipart()
# message['From'] = sender_address
# message['To'] = receiver_address
# message['Subject'] = 'A test mail with attachment sent by local Zoidberg.'

# message.attach(MIMEText(mail_content, 'plain'))
# attach_file_name = 'myfile.xlsx'
# with open(attach_file_name, 'rb') as attach_file:# Open the file as binary mode
#     part = MIMEBase('application', "octet-stream")
#     part.set_payload((attach_file).read())
#     encoders.encode_base64(part)

# payload.add_header('Content-Disposition', 'attachment', filename=attach_file_name)
# message.attach(payload)

# session = smtplib.SMTP('casmail.agoda.local', 25) #use agoda mail with port
# text = message.as_string()
# session.sendmail(sender_address, receiver_address, text)
# session.close()
# print('Mail Sent')

start download from /user/hk-pdops-svc/Tools/LUNA/myfile.xlsx to ./myfile.xlsx
downloaded ./myfile.xlsx


# Exception

In [12]:
# hdfs_client.download_file(historical_f, './input.xlsx')
# user_input = openpyxl.load_workbook('input.xlsx')
# sheet = user_input.active
# maxRow = sheet.max_row

start download from /user/hk-pdops-svc/Tools/LUNA/input.xlsx to ./input.xlsx
downloaded ./input.xlsx
